<a href="https://colab.research.google.com/github/mohit25bai11111/ASL-Recognition-AI-Model/blob/main/ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile

# ==========================================
# 1. CHECK IF ZIP EXISTS
# ==========================================
zip_path = "asl-alphabet.zip"

if not os.path.exists(zip_path):
    print("❌ ERROR: 'asl-alphabet.zip' file is missing!")
    print("--- SOLUTION ---")
    print("1. Did you upload kaggle.json?")
    print("2. If the download failed, you can manually download the zip from Kaggle")
    print("   and drag-and-drop it into the Files sidebar on the left.")
else:
    print(f"✅ Found {zip_path}")

    # ==========================================
    # 2. FORCE UNZIP (Overwrite everything)
    # ==========================================
    print("Unzipping data... (Please wait 1-2 mins)")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("./")
    print("✅ Unzip complete.")

# ==========================================
# 3. SEARCH FOR THE DATA FOLDER
# ==========================================
print("Searching for the 'A' folder...")

found_data_dir = None

# Walk through every folder in the current directory
for root, dirs, files in os.walk("."):
    # We look for a folder that contains 'A', 'B', and 'Z'
    if "A" in dirs and "B" in dirs and "Z" in dirs:
        found_data_dir = root
        break

if found_data_dir:
    print(f"\n🎉 SUCCESS! Found dataset at: {found_data_dir}")
    print(f"Variable DATA_DIR is now set to: '{found_data_dir}'")

    # SET THE GLOBAL VARIABLE
    DATA_DIR = found_data_dir

    # VERIFY BY LISTING FILES
    print("Classes found:", os.listdir(DATA_DIR)[:5])
else:
    print("\n❌ STILL FAILED: Could not find the folders 'A', 'B', 'Z'.")
    print("Please check the 'Files' sidebar on the left to see what happened.")

✅ Found asl-alphabet.zip
Unzipping data... (Please wait 1-2 mins)
✅ Unzip complete.
Searching for the 'A' folder...

🎉 SUCCESS! Found dataset at: ./asl_alphabet_train/asl_alphabet_train
Variable DATA_DIR is now set to: './asl_alphabet_train/asl_alphabet_train'
Classes found: ['R', 'V', 'D', 'C', 'M']


In [ ]:
import os
# Print ALL classes, sorted alphabetically
print(sorted(os.listdir(DATA_DIR)))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 1. CHECK EXACT CLASS COUNT
num_classes = len(train_generator.class_indices)
print(f"🧐 Generator found {num_classes} classes.")

if num_classes == 0:
    print("❌ CRITICAL ERROR: Generator found 0 classes. Check your DATA_DIR path!")
else:
    print("🧠 Building MobileNetV2 Model to match...")

    # 2. Load Base Model
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base_model.trainable = False

    # 3. Add Custom Layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    # --- THE FIX IS HERE ---
    # We use 'num_classes' instead of hardcoding 29
    predictions = Dense(num_classes, activation='softmax')(x)

    # 4. Create Model
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print(f"✅ Model Built! Output layer set to {num_classes} neurons.")
    print("🚀 Starting Training...")

    # 5. Train
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator,
        verbose=1
    )

    print("🎉 Training Complete!")

🧐 Generator found 13 classes.
🧠 Building MobileNetV2 Model to match...
✅ Model Built! Output layer set to 13 neurons.
🚀 Starting Training...
Epoch 1/5
463/463 ━━━━━━━━━━━━━━━━━━━━ 961s 2s/step - accuracy: 0.8492 - loss: 0.5315 - val_accuracy: 0.9281 - val_loss: 0.2335
Epoch 2/5
463/463 ━━━━━━━━━━━━━━━━━━━━ 939s 2s/step - accuracy: 0.9958 - loss: 0.0230 - val_accuracy: 0.9512 - val_loss: 0.1604
Epoch 3/5
463/463 ━━━━━━━━━━━━━━━━━━━━ 936s 2s/step - accuracy: 0.9979 - loss: 0.0101 - val_accuracy: 0.9268 - val_loss: 0.2598
Epoch 4/5
463/463 ━━━━━━━━━━━━━━━━━━━━ 1000s 2s/step - accuracy: 0.9989 - loss: 0.0064 - val_accuracy: 0.9571 - val_loss: 0.1464
Epoch 5/5
463/463 ━━━━━━━━━━━━━━━━━━━━ 990s 2s/step - accuracy: 0.9993 - loss: 0.0044 - val_accuracy: 0.9406 - val_loss: 0.2153
🎉 Training Complete!


In [12]:
import pickle
from google.colab import files

print("💾 Saving Model...")
# 1. Save the trained brain
model.save('asl_mobile_final.h5')

print("💾 Saving Dictionary...")
# 2. Save the list of letters (A=0, B=1, etc.)
# We need this so the AI knows that '0' means 'A'
with open('asl_indices.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)

print("⬇️ Downloading files to your laptop...")
files.download('asl_mobile_final.h5')
files.download('asl_indices.pkl')


💾 Saving Model...
💾 Saving Dictionary...
⬇️ Downloading files to your laptop...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>